In [68]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [69]:
print("NumPy version " + np.__version__)
print("Tensorflow version " + tf.__version__)

NumPy version 1.18.5
Tensorflow version 2.3.1


In [70]:
## Data Reading ##
#tf.data.experimental.CsvDataset("data2/train_drug.csv", "float32")
train_features = pd.read_csv("data/train_features.csv")
results = pd.read_csv("data/train_targets_scored.csv")
testSet = pd.read_csv("data/test_features.csv")

In [71]:
## Parameters ##
validationProportion = 0.2

N = len(train_features) #Nombre d'échantillons testés
M = testSet.shape[1]-1 #Nombre de pathologies obserables

In [72]:
## Normalization ##
maxVal=np.max(np.max(np.abs(train_features.iloc[:,4:])))
train_features.iloc[:,4:] = train_features.iloc[:,4:]/maxVal
train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,0.10620,0.05577,-0.02479,-0.06208,-0.01944,-0.10120,...,0.02862,0.02584,0.08076,0.05523,-0.01912,0.06584,-0.03981,0.02139,0.03801,0.04176
1,id_000779bfc,trt_cp,72,D1,0.00743,0.04087,0.02991,0.00604,0.10190,0.05207,...,-0.04265,0.07543,0.04708,0.00230,0.02957,0.04899,0.01522,0.01241,0.06077,0.07371
2,id_000a6266a,trt_cp,48,D1,0.06280,0.05817,0.15540,-0.00764,-0.00323,0.12390,...,-0.07250,-0.06297,0.06103,0.00223,-0.13240,-0.03174,-0.06417,-0.02187,-0.14080,0.06931
3,id_0015fd391,trt_cp,48,D1,-0.05138,-0.02491,-0.02656,0.05288,0.40620,-0.08095,...,-0.20990,-0.06441,-0.56300,-0.13780,-0.08632,-0.12880,-0.16210,-0.08784,-0.03876,-0.08154
4,id_001626bd3,trt_cp,72,D2,-0.03254,-0.04009,0.09700,0.06919,0.14180,-0.08244,...,0.00042,0.00048,0.06670,0.10690,0.05523,-0.03031,0.01094,0.02885,-0.03786,0.07125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.01394,-0.00636,-0.01112,-0.05080,-0.04713,0.07201,...,0.01969,0.00262,-0.08121,0.03434,0.05372,-0.03246,0.00631,0.09171,0.05258,0.04680
23810,id_fffb70c0c,trt_cp,24,D2,-0.13260,0.03478,-0.03743,0.09905,-0.07178,0.06621,...,0.04286,0.04426,0.00423,-0.03195,-0.08086,-0.09798,-0.02084,-0.01224,-0.02715,0.03689
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.03942,0.03756,0.03109,-0.07389,0.05505,-0.00159,...,0.05409,0.03755,0.07343,0.02807,0.04116,0.06422,0.02256,0.07592,0.06656,0.03808
23812,id_fffcb9e7c,trt_cp,24,D1,0.06660,0.02324,0.04392,0.02044,0.08531,-0.00343,...,-0.01105,0.04258,-0.02012,0.01506,0.15230,0.07101,0.01732,0.07015,-0.06290,0.00740


In [73]:
## replacing sig_id by drug_id ##
train_features = train_features.join(results, lsuffix='sig_id', rsuffix='sig_id')
train_features = train_features.drop(columns=['sig_idsig_id'])

train_features

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,trt_cp,24,D1,0.10620,0.05577,-0.02479,-0.06208,-0.01944,-0.10120,-0.10220,...,0,0,0,0,0,0,0,0,0,0
1,trt_cp,72,D1,0.00743,0.04087,0.02991,0.00604,0.10190,0.05207,0.02341,...,0,0,0,0,0,0,0,0,0,0
2,trt_cp,48,D1,0.06280,0.05817,0.15540,-0.00764,-0.00323,0.12390,0.01715,...,0,0,0,0,0,0,0,0,0,0
3,trt_cp,48,D1,-0.05138,-0.02491,-0.02656,0.05288,0.40620,-0.08095,-0.19590,...,0,0,0,0,0,0,0,0,0,0
4,trt_cp,72,D2,-0.03254,-0.04009,0.09700,0.06919,0.14180,-0.08244,-0.02800,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,trt_cp,24,D2,0.01394,-0.00636,-0.01112,-0.05080,-0.04713,0.07201,0.05773,...,0,0,0,0,0,0,0,0,0,0
23810,trt_cp,24,D2,-0.13260,0.03478,-0.03743,0.09905,-0.07178,0.06621,-0.02252,...,0,0,0,0,0,0,0,0,0,0
23811,ctl_vehicle,48,D2,0.03942,0.03756,0.03109,-0.07389,0.05505,-0.00159,-0.02541,...,0,0,0,0,0,0,0,0,0,0
23812,trt_cp,24,D1,0.06660,0.02324,0.04392,0.02044,0.08531,-0.00343,0.00323,...,0,0,0,0,0,0,0,0,0,0


In [74]:
## Shuffling ##
train_features = train_features.reindex(np.random.permutation(train_features.index))
train_features

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
19116,trt_cp,48,D1,0.23040,0.05918,0.03664,-0.14050,0.00500,0.07069,0.06898,...,0,0,0,0,0,0,0,0,0,0
14002,trt_cp,24,D2,0.00119,-0.07487,0.00762,0.07340,-0.02424,0.03845,-0.03114,...,0,0,0,0,0,0,0,0,0,0
4758,trt_cp,72,D1,-0.03139,0.06424,-0.01042,-0.05386,0.02428,-0.23310,0.02941,...,0,0,0,0,0,0,0,0,0,0
7669,trt_cp,24,D1,0.48610,0.08569,0.10380,0.00974,0.28240,-0.17530,-0.03517,...,0,0,0,0,0,0,0,0,0,0
17193,trt_cp,24,D1,0.15380,-0.28580,0.13950,0.18270,0.12870,-0.00136,0.06703,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5738,trt_cp,48,D2,0.00893,0.03346,-0.00485,0.03417,-0.02411,-0.00053,-0.03541,...,0,0,0,0,0,0,0,0,0,0
18436,trt_cp,48,D1,0.11720,-0.02436,-0.04600,-0.01757,-0.09099,-0.02690,0.03826,...,0,0,0,1,0,0,0,0,0,0
7996,trt_cp,48,D1,-0.12380,-0.03111,-0.06927,0.00071,0.04701,0.05791,-0.06015,...,0,0,0,1,0,0,0,0,0,0
18600,ctl_vehicle,24,D2,-0.05123,0.00937,-0.04576,0.18330,0.03417,-0.00551,0.19590,...,0,0,0,0,0,0,0,0,0,0


In [75]:
## Replacing categories values by numbers ##
features_to_convert=['cp_type','cp_time','cp_dose']
for feat in features_to_convert:
    train_features[feat] = pd.Categorical(train_features[feat])
    train_features[feat] = train_features[feat].cat.codes
    
train_features

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
19116,1,1,0,0.23040,0.05918,0.03664,-0.14050,0.00500,0.07069,0.06898,...,0,0,0,0,0,0,0,0,0,0
14002,1,0,1,0.00119,-0.07487,0.00762,0.07340,-0.02424,0.03845,-0.03114,...,0,0,0,0,0,0,0,0,0,0
4758,1,2,0,-0.03139,0.06424,-0.01042,-0.05386,0.02428,-0.23310,0.02941,...,0,0,0,0,0,0,0,0,0,0
7669,1,0,0,0.48610,0.08569,0.10380,0.00974,0.28240,-0.17530,-0.03517,...,0,0,0,0,0,0,0,0,0,0
17193,1,0,0,0.15380,-0.28580,0.13950,0.18270,0.12870,-0.00136,0.06703,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5738,1,1,1,0.00893,0.03346,-0.00485,0.03417,-0.02411,-0.00053,-0.03541,...,0,0,0,0,0,0,0,0,0,0
18436,1,1,0,0.11720,-0.02436,-0.04600,-0.01757,-0.09099,-0.02690,0.03826,...,0,0,0,1,0,0,0,0,0,0
7996,1,1,0,-0.12380,-0.03111,-0.06927,0.00071,0.04701,0.05791,-0.06015,...,0,0,0,1,0,0,0,0,0,0
18600,0,0,1,-0.05123,0.00937,-0.04576,0.18330,0.03417,-0.00551,0.19590,...,0,0,0,0,0,0,0,0,0,0


In [78]:
## Splitting into training and validation ##
trainingSize = int((1-validationProportion) * N)

trainingSet = train_features.iloc[:trainingSize,:] 
validationSet = train_features.iloc[trainingSize:,:]

print("Training size:", len(trainingSet))
print("Validation size:", len(validationSet))

Training size: 19051
Validation size: 4763


In [77]:
## Batching ##
"""
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), dataframe))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

trainingds = df_to_dataset(trainingSet)
trainingds
"""

'\n# A utility method to create a tf.data dataset from a Pandas Dataframe\ndef df_to_dataset(dataframe, shuffle=True, batch_size=32):\n    dataframe = dataframe.copy()\n    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), dataframe))\n    if shuffle:\n        ds = ds.shuffle(buffer_size=len(dataframe))\n    ds = ds.batch(batch_size)\n    ds = ds.prefetch(batch_size)\n    return ds\n\ntrainingds = df_to_dataset(trainingSet)\ntrainingds\n'